# DAY-2

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
path = "day1_cleaned.csv"
df = pd.read_csv(path)
print(df.head())
print(df.info())

                                      company  \
0                            MM Media Pvt Ltd   
1                          find live infotech   
2         Softtech Career Infosystem Pvt. Ltd   
3                      Onboard HRServices LLP   
4  Spire Technologies and Solutions Pvt. Ltd.   

                                           education   experience  \
0  UG: B.Tech/B.E. - Any Specialization PG:Any Po...    0 - 1 yrs   
1  UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...    0 - 0 yrs   
2  UG: Any Graduate - Any Specialization PG:Any P...    4 - 8 yrs   
3  UG: Any Graduate - Any Specialization PG:CA Do...  11 - 15 yrs   
4  UG: B.Tech/B.E. - Any Specialization PG:Any Po...    6 - 8 yrs   

                                   industry  \
0          Media / Entertainment / Internet   
1  Advertising / PR / MR / Event Management   
2           IT-Software / Software Services   
3    Banking / Financial Services / Broking   
4           IT-Software / Software Services   

       

In [94]:
df.rename(columns = {'joblocation_address': 'location',
                    'jobtitle': 'job_title',
                    'jobdescription': 'description',
                    'numberofpositions': 'num_positions',
                    'payrate': 'salary',
                    'postdate': 'date_posted',
                    'site_name': 'source',
                    'uniq_id': 'id'}, inplace=True)

## Let’s clean job titles step by step so they’re normalized and grouped properly.

In [95]:
df["job_title_cleaned"] = df["job_title"].str.lower()
df["job_title_Cleaned"] = df["job_title_cleaned"].str.replace(r'[^a-z0-9\s]', '', regex=True)
df["job_title_cleaned"] = df["job_title_cleaned"].str.strip()

In [96]:
data_scientist_jobs = df['job_title_cleaned'][df['job_title_cleaned'].str.startswith('data scientist', na=False)]
print(data_scientist_jobs.unique())


['data scientist / bangalore'
 'data scientist/vba programming/r programming/fmcg-cpg/bangalore'
 'data scientist - machine learning'
 'data scientist partners for start-up' 'data scientist'
 'data scientist-machine learning' 'data scientist big data ml'
 'data scientist (machine learning)' 'data scientist machine learning'
 'data scientist - bangalore' 'data scientist - telecom domain'
 'data scientist senior data scientist'
 'data scientist/algorithm model developer'
 'data scientist - machine learning/nlp'
 'data scientist - data modelling - forecasting'
 'data scientist (big data)' 'data scientist (bangalore)']


## Let’s automatically normalize all “Data Scientist” variants in your dataset. 
### We’ll do this in a systematic way

In [97]:
ds_titles_mask  = df["job_title_cleaned"].str.contains("data scientist", na=False)
ds_varitaions = df.loc[ds_titles_mask, "job_title_cleaned"].unique()
print("All 'Data scientist' variations: \n", ds_varitaions)

All 'Data scientist' variations: 
 ['lead data scientist' 'data scientist / bangalore'
 'data scientist/vba programming/r programming/fmcg-cpg/bangalore'
 'data scientist - machine learning'
 'data scientist partners for start-up' 'data scientist'
 'director - data scientist - nlp/r/sas' 'senior data scientist'
 'data scientist-machine learning' 'big data scientist'
 'data scientist big data ml' 'excellent opportunity for data scientist'
 'nlp data scientist' 'data scientist (machine learning)'
 'principal data scientist' 'data scientist machine learning'
 'data scientist - bangalore' 'data scientist - telecom domain'
 'senior data scientist / algorithms specialist'
 'data scientist senior data scientist' 'senior engineer data scientist'
 'chief data scientist' 'data scientist/algorithm model developer'
 'data scientist - machine learning/nlp'
 'data scientist - data modelling - forecasting'
 'principle statistician / data scientist' 'data scientist (big data)'
 'principle statistician

In [98]:
ds_mapping = {title: "data scientist" for title in ds_varitaions}
df['job_title_cleaned'] = df['job_title_cleaned'].replace(ds_mapping)

In [99]:
# Check unique job titles again
print(df['job_title_cleaned'].value_counts().head(20))

job_title_cleaned
business development manager                                              101
business development executive                                             96
software engineer                                                          88
android developer                                                          77
php developer                                                              72
project manager                                                            71
web designer                                                               69
content writer                                                             68
java developer                                                             61
sales executive                                                            60
senior software engineer                                                   60
dot net developer                                                          55
marketing executive                           

In [100]:

# Check only Data Scientist now
print(df[df['job_title_cleaned'] == 'data scientist'].shape[0])


40


In [101]:
data_scientist_jobs = df['job_title_cleaned'][df['job_title_cleaned'].str.startswith('data scientist', na=False)]
print(data_scientist_jobs.unique())

['data scientist']


# Result
All senior, junior, lead, or other variations of Data Scientist are now grouped under data scientist.
Makes aggregation and salary analysis much cleaner.
we can repeat this for other roles like Software Engineer, Frontend Engineer, Backend Engineer, etc., by changing the keyword in str.contains().

In [102]:
# print(df["job_title"].unique())
# print(df["job_title"].value_counts())
# job_title_counts = df['job_title'].value_counts()

# job_title_counts.to_csv("job_title_counts.csv", header=True)


In [125]:

import pandas as pd
import re

# 1️⃣ Normalize text: lowercase, remove symbols, strip spaces
df['job_title_cleaned'] = df['job_title'].str.lower()
df['job_title_cleaned'] = df['job_title_cleaned'].str.replace(r'[^a-z0-9\s]', '', regex=True)
df['job_title_cleaned'] = df['job_title_cleaned'].str.strip()

# 2️⃣ Remove senior/junior/lead/principal/associate prefixes
seniority_words = ['sr', 'senior', 'jr', 'junior', 'lead', 'principal', 'associate', 'trainee']
pattern = r'\b(?:' + '|'.join(seniority_words) + r')\b'
df['job_title_cleaned'] = df['job_title_cleaned'].str.replace(pattern, '', regex=True)
df['job_title_cleaned'] = df['job_title_cleaned'].str.replace(r'\s+', ' ', regex=True).str.strip()  # remove extra spaces

# 3️⃣ Remove numbers, emails, and contact info
df['job_title_cleaned'] = df['job_title_cleaned'].str.replace(r'\d+', '', regex=True)  # remove digits
df['job_title_cleaned'] = df['job_title_cleaned'].str.replace(r'\S+@\S+', '', regex=True)  # remove emails
df['job_title_cleaned'] = df['job_title_cleaned'].str.strip()

# 4️⃣ Truncate very long titles to first 6 words (optional)
df['job_title_cleaned'] = df['job_title_cleaned'].apply(lambda x: ' '.join(x.split()[:6]))

# 5️⃣ Keyword mapping: normalize common roles
role_keywords = {
    'data scientist': 'Data Scientist',
    'machine learning': 'Data Scientist',
    'ai engineer': 'Data Scientist',
    'software engineer': 'Software Engineer',
    'developer': 'Software Engineer',
    'full stack': 'Full Stack Engineer',
    'frontend': 'Frontend Engineer',
    'backend': 'Backend Engineer',
    'ios': 'iOS Developer',
    'android': 'Android Developer',
    'web designer': 'Web Designer',
    'graphic designer': 'Graphic Designer',
    'ui': 'UI Developer',
    'php': 'PHP Developer',
    'java': 'Java Developer',
    'dot net': '.NET Developer',
    'project manager': 'Project Manager',
    'product manager': 'Product Manager',
    'business development': 'Business Development',
    'business analyst': 'Business Analyst',
    'marketing': 'Marketing',
    'sales': 'Sales',
    'hr': 'HR',
    'accountant': 'Accountant',
    'devops': 'DevOps Engineer',
    'network': 'Network Engineer',
    'technical writer': 'Technical Writer',
    'system administrator': 'System Administrator',
    'analyst': 'Analyst',
    'associate consultant': 'Associate Consultant',
    'quality assurance': 'Quality Assurance'
    # Add more keywords if needed
}

# 6️⃣ Executive-level mapping
executive_keywords = ['chief', 'cto', 'ceo', 'cfo', 'vp', 'director', 'head', 'gm', 'dgm', 'leader']
def map_executive(title):
    for keyword in executive_keywords:
        if keyword in title:
            return 'Executive / Director'
    for keyword, standard in role_keywords.items():
        if keyword in title:
            return standard
    return title.title()  # fallback


df['job_title_cleaned'] = df['job_title_cleaned'].apply(map_job_title)



In [126]:

# 6️⃣ Verify results
print(df[['job_title', 'job_title_cleaned']].head(20))
print(df['job_title_cleaned'].value_counts().head(20))


                                            job_title  \
0            Walkin Data Entry Operator (night Shift)   
1                  Work Based Onhome Based Part Time.   
2                              Pl/sql Developer - SQL   
3              Manager/ad/partner - Indirect Tax - CA   
4                     JAVA Technical Lead (6-8 yrs) -   
5   WALK IN - As400 Developer - Pfsweb Global Serv...   
6                                       PHP Developer   
7   Member Technical Staff-wire Harness/cable Harn...   
8                                         Team Leader   
9                                   German Translator   
10                       Business Development Manager   
11    Opening for Android Developer-bangalore-4-8 yrs   
12         Full Stack Web Application (php) Developer   
13           Looking for an Application Engineer-fpga   
14                                  Revenue Assurance   
15  Senior Java Full Stack Developer - Java/ Spring 4   
16                             

In [127]:
print(df['job_title_cleaned'].value_counts().tail(30))
# print(df['job_title'].value_counts().tail(30))

job_title_cleaned
Associatesr Transaction Support Ca Big                      1
Office Assistant Computer Operator                          1
Consultantmanager Document Review Lpo Llbllm                1
Manager Collection Nbfc                                     1
Job Schedulers Technical Design Architecture                1
Faculty Positions Ecommerce                                 1
Immediate Opening For Sql Dba Pune                          1
Tester Delhi                                                1
Project Finance Fund Raising Delhi South                    1
Commodity Broker                                            1
Manager Polyethylene Films                                  1
Chinese Chef Starters Soups Sauces Dimsums                  1
Image Processing Algorithm Development Adas Applications    1
Mega Walkins For Education Counsellor Executive             1
Eeg Engineer                                                1
Group Manager Fundamental Research New Search       

In [128]:
job_title_counts = df['job_title_cleaned'].value_counts()

job_title_counts.to_csv("job_title_counts.csv", header=True)


In [129]:
print(df.columns)

Index(['company', 'education', 'experience', 'industry', 'description',
       'jobid', 'location', 'job_title', 'num_positions', 'salary',
       'date_posted', 'source', 'skills', 'id', 'salary_cleaned',
       'job_title_cleaned'],
      dtype='object')


In [132]:
avg_salary = df.groupby('job_title_cleaned')['salary_cleaned'].mean().sort_values(ascending=False)
print(avg_salary.head(15))

job_title_cleaned
Mobile Solution Architects                              5000000.0
Business Headindustrial Consumablemncchennai            5000000.0
Brand Director                                          5000000.0
Dgm Gm Govt Regulatory Affairs Smartphone               5000000.0
Chief Technology Officer                                5000000.0
Unica                                                   5000000.0
Immediate Job Opening Sharepoint                        5000000.0
Director Director Development Technical Director        4750000.0
Njs Consultant Interventional Cardiologist Hyderabad    4500000.0
Plant Head Manager Beverages Brewery Tins               4500000.0
Compliance Leader Ecommerce Digital                     4500000.0
Ibm San Sme Yrs Exp Ibm                                 4250000.0
Vp Facility Management Mnc Real Estate                  4250000.0
Leader Learning Organization Development It             4250000.0
Technical Architect For An Ecommerce Mnc                42

In [131]:
df.to_csv("day2_cleaned.csv", index=False)

In [133]:
!git add .
    

In [134]:
!git commit -m "Drop duplicate"

[main 8b05b6b] Drop duplicate
 2 files changed, 22114 insertions(+), 22117 deletions(-)


In [135]:
!git push

Enumerating objects: 19, done.
Counting objects: 100% (19/19), done.
Delta compression using up to 12 threads
Compressing objects: 100% (13/13), done.
Writing objects: 100% (13/13), 723.78 KiB | 1011.00 KiB/s, done.
Total 13 (delta 9), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (9/9), completed with 5 local objects.
remote: warning: See https://gh.io/lfs for more information.
remote: warning: File Week -1/day2_cleaned.csv is 51.00 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: File Week -1/day2_cleaned.csv is 50.30 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
To https://github.com/palaknagar-07/CareerPlus.git
   44e859b..8b05b6b  main -> main
